<a href="https://colab.research.google.com/github/vpuzio-deloitte/digita/blob/main/esempio_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install langchain_core langchain_google_genai boto3 botocore langchain_aws python-docx PyMuPDF langchain_community chromadb

## Init LLM

In [19]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm_bedrock = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )

## Utilities

In [20]:
import fitz  # PyMuPDF
import docx

from google.colab import files

def upload_file():
  uploaded = files.upload()
  # Una volta eseguita questa cella, Colab ti chiederà di selezionare il file dal tuo PC.

  # 2) Identifica il nome del file caricato
  for file_name in uploaded.keys():
      file_path = file_name

  return file_path

def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text

def read_txt(file_path):
    with open(file_path, "r") as f:
        text = f.read()
    return text

def upload_and_read_pdf():
    file_path = upload_file()

    return read_pdf(file_path)

def upload_and_read_docx():
    file_path = upload_file()

    return read_docx(file_path)

def upload_and_read_txt():
    file_path = upload_file()

    return read_txt(file_path)

## Upload file and split

In [7]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Carica il contenuto dal file (es. "faq.txt")
text = upload_and_read_txt()

# 2. Chunkizza il testo per paragrafo
#    Si assume che i paragrafi siano separati da due newline ("\n\n")
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=500,     # Dimensione massima di ciascun chunk
    chunk_overlap=50    # Sovrapposizione tra chunk per mantenere il contesto
)
chunks = text_splitter.split_text(text)

# Trasforma ciascun chunk in un oggetto Document
documents = [Document(page_content=chunk) for chunk in chunks]


Saving rag.txt to rag (5).txt


## Vector store

In [8]:
from langchain.llms.anthropic import Anthropic
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
from langchain.vectorstores import Chroma

# 3. Inizializza il provider di embeddings utilizzando BedrockEmbeddings
#    Specifica il modello e la regione AWS in cui è disponibile Bedrock.
#    Consulta la documentazione per impostare eventuali parametri aggiuntivi.
bedrock_embeddings = BedrockEmbeddings(
    client=bedrock_client,
    model_id="amazon.titan-embed-text-v2:0",  # Esempio: sostituisci con l'ID del modello desiderato
    region_name="eu-west-1"         # Sostituisci con la regione corretta
)

# 4. Crea il vector store con Chroma utilizzando i documenti e gli embeddings di Bedrock
vectorstore = Chroma.from_documents(
    documents,
    bedrock_embeddings,
    collection_name="faq_collection"  # Nome della collezione
)

# 5. Configura il retriever per ottenere, ad esempio, i 3 documenti più rilevanti per ogni query
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


## Retrieval

In [13]:
from langchain.chains import RetrievalQA

# 7. Crea la catena RetrievalQA che integra il retriever e il modello LLM
qa_chain = RetrievalQA.from_chain_type(llm=llm_bedrock, chain_type="stuff", retriever=retriever)

# 8. Esegui una query per ottenere una risposta in modalità RAG
query = "Come posso risolvere il problema della connessione a Internet?"
result = qa_chain.invoke(query)


Risposta: {'query': 'Come posso risolvere il problema della connessione a Internet?', 'result': 'Sulla base delle informazioni fornite, ecco alcuni suggerimenti per risolvere il problema di connessione a Internet:\n\n- Verifica la configurazione del tuo dispositivo: assicurati che le impostazioni di rete siano corrette e che il dispositivo sia configurato correttamente per connettersi a Internet.\n\n- Riavvia il dispositivo e il router/modem: spesso un semplice riavvio può risolvere problemi temporanei di connessione.\n\n- Controlla il cavo Ethernet o la connessione Wi-Fi: verifica che il cavo sia inserito correttamente e che il segnale Wi-Fi sia presente e forte.\n\n- Verifica eventuali interruzioni del servizio: controlla sul sito web o sui canali social del tuo provider se ci sono segnalazioni di problemi di rete nella tua zona.\n\n- Contatta il servizio clienti del tuo provider: se i passaggi precedenti non risolvono il problema, contatta il supporto tecnico del tuo provider di Int

## Display output

In [18]:
from IPython.display import display, Markdown
import json

display(Markdown(f"### Riassunto:\n\n{result['result']}"))


### Riassunto:

Sulla base delle informazioni fornite, ecco alcuni suggerimenti per risolvere il problema di connessione a Internet:

- Verifica la configurazione del tuo dispositivo: assicurati che le impostazioni di rete siano corrette e che il dispositivo sia configurato correttamente per connettersi a Internet.

- Riavvia il dispositivo e il router/modem: spesso un semplice riavvio può risolvere problemi temporanei di connessione.

- Controlla il cavo Ethernet o la connessione Wi-Fi: verifica che il cavo sia inserito correttamente e che il segnale Wi-Fi sia presente e forte.

- Verifica eventuali interruzioni del servizio: controlla sul sito web o sui canali social del tuo provider se ci sono segnalazioni di problemi di rete nella tua zona.

- Contatta il servizio clienti del tuo provider: se i passaggi precedenti non risolvono il problema, contatta il supporto tecnico del tuo provider di Internet, che potrà aiutarti a diagnosticare e risolvere il problema.

Se non riesci a risolvere il problema in autonomia, il supporto tecnico del tuo provider sarà in grado di fornire ulteriore assistenza e individuare la causa del problema di connessione.